# Movie Recommendation
## using K-Nearest Neighbours

This is a Jupyter Notebook to develop a movies recommendation system using K-Nearest Neighbous.

The dataset used is [MovieLens Dataset](https://grouplens.org/datasets/movielens/latest/). The *Small* dataset is being used here for educational and development purposes.

In [1]:
import pandas as pd
import numpy as np

## Importing Datasets and creating DataFrames

In [2]:
movies = pd.read_csv('data/movies_data/movies.csv')
ratings = pd.read_csv('data/movies_data/ratings.csv')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


### Preparing Dataset

In [5]:
print('Movies df Shape: ', movies.shape)
print('Ratings df Shape: ', ratings.shape)

Movies df Shape:  (9742, 3)
Ratings df Shape:  (100836, 4)


In [6]:
df = ratings.merge(movies, on='movieId')

In [7]:
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [8]:
rating_counts = ratings.groupby('movieId')['rating'].count().sort_values(ascending=False)
rating_counts.head()

movieId
356     329
318     317
296     307
593     279
2571    278
Name: rating, dtype: int64

In [9]:
movies = movies.merge(rating_counts, on='movieId')
movies.rename(columns={'rating':'rating_count'}, inplace=True)
movies

,movieId,title,genres,rating_count
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
1,2,Jumanji (1995),Adventure|Children|Fantasy,110
2,3,Grumpier Old Men (1995),Comedy|Romance,52
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,7
4,5,Father of the Bride Part II (1995),Comedy,49
...,...,...,...,...
9719,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,1
9720,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,1
9721,193585,Flint (2017),Drama,1
9722,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,1


In [10]:
movies[movies['movieId']==356]

,movieId,title,genres,rating_count
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War,329


In [11]:
rating_avg = ratings.groupby('movieId')['rating'].mean().sort_values(ascending=False)
rating_avg.head()

movieId
88448     5.0
100556    5.0
143031    5.0
143511    5.0
143559    5.0
Name: rating, dtype: float64

In [12]:
movies = movies.merge(rating_avg, on='movieId')
movies.rename(columns={'rating':'rating_avg'}, inplace=True)
movies

,movieId,title,genres,rating_count,rating_avg
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215,3.920930
1,2,Jumanji (1995),Adventure|Children|Fantasy,110,3.431818
2,3,Grumpier Old Men (1995),Comedy|Romance,52,3.259615
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,7,2.357143
4,5,Father of the Bride Part II (1995),Comedy,49,3.071429
...,...,...,...,...,...
9719,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,1,4.000000
9720,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,1,3.500000
9721,193585,Flint (2017),Drama,1,3.500000
9722,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,1,3.500000


### Calculating Weighted Average Rating of a Movie

Formula Used:  
&emsp; &emsp; *w = (Rv + Cm) / (v + m)*   
  
where,  
&emsp; w = weighted rating  
&emsp; R = average rating of the movie  
&emsp; v = number of total votes (ratings)  
&emsp; m = minimum votes to be listed in top 75% of the data  
&emsp; C = mean across all the average ratings

In [13]:
v = movies['rating_count']
R = movies['rating_avg']
m = movies['rating_count'].quantile(q=0.75)
C = movies['rating_avg'].mean()

In [14]:
movies['weighted_avg'] = ((R * v) + (C * m)) / (v + m) 

In [15]:
movies.head()

,movieId,title,genres,rating_count,rating_avg,weighted_avg
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215,3.920930,3.894473
1,2,Jumanji (1995),Adventure|Children|Fantasy,110,3.431818,3.419009
2,3,Grumpier Old Men (1995),Comedy|Romance,52,3.259615,3.260033
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,7,2.357143,2.866377
4,5,Father of the Bride Part II (1995),Comedy,49,3.071429,3.101070


### Handling Genres Feature

In [16]:
genres = movies['genres']
genres.head()

0    Adventure|Animation|Children|Comedy|Fantasy
1                     Adventure|Children|Fantasy
2                                 Comedy|Romance
3                           Comedy|Drama|Romance
4                                         Comedy
Name: genres, dtype: object

In [17]:
unique_genres = []
for movie in genres:
    list_genre = movie.split('|')
    for item in list_genre:
        if item not in unique_genres:
            unique_genres.append(item)
unique_genres

['Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Fantasy',
 'Romance',
 'Drama',
 'Action',
 'Crime',
 'Thriller',
 'Horror',
 'Mystery',
 'Sci-Fi',
 'War',
 'Musical',
 'Documentary',
 'IMAX',
 'Western',
 'Film-Noir',
 '(no genres listed)']

In [18]:
for genre in unique_genres:
    movies[genre] = 0

In [19]:
movies.head()

,movieId,title,genres,rating_count,rating_avg,weighted_avg,Adventure,Animation,Children,Comedy,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215,3.920930,3.894473,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,110,3.431818,3.419009,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,52,3.259615,3.260033,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,7,2.357143,2.866377,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,49,3.071429,3.101070,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
for index, list_genre in enumerate(genres):
    list_genre_split = list_genre.split('|')
    for item in list_genre_split:
        movies.at[index, item] = 1
movies.head()

,movieId,title,genres,rating_count,rating_avg,weighted_avg,Adventure,Animation,Children,Comedy,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215,3.920930,3.894473,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,110,3.431818,3.419009,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,52,3.259615,3.260033,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,7,2.357143,2.866377,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,49,3.071429,3.101070,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [21]:
movies.drop(['(no genres listed)'], inplace=True, axis=1)
movies.head()

,movieId,title,genres,rating_count,rating_avg,weighted_avg,Adventure,Animation,Children,Comedy,...,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215,3.920930,3.894473,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,110,3.431818,3.419009,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,52,3.259615,3.260033,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,7,2.357143,2.866377,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,49,3.071429,3.101070,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


### Creating Model to Dump Later

In [22]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(np.array(movies['rating_count']).reshape(-1, 1))

StandardScaler()

In [23]:
movies['rating_count'] = (movies['rating_count'] - movies['rating_count'].mean()) / movies['rating_count'].std()

In [24]:
movies.head()

,movieId,title,genres,rating_count,rating_avg,weighted_avg,Adventure,Animation,Children,Comedy,...,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9.134867,3.920930,3.894473,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,4.447577,3.431818,3.419009,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,1.858407,3.259615,3.260033,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,-0.150431,2.357143,2.866377,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,1.724485,3.071429,3.101070,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [25]:
X = movies.drop(['movieId', 'title', 'genres'], axis=1)
movies_data = movies[['movieId', 'title', 'genres']]

In [26]:
X.head()

,rating_count,rating_avg,weighted_avg,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir
0,9.134867,3.920930,3.894473,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4.447577,3.431818,3.419009,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.858407,3.259615,3.260033,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,-0.150431,2.357143,2.866377,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1.724485,3.071429,3.101070,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
movies_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [28]:
X.to_csv('data/source/movies_encoded_data.csv', index=False)
movies_data.to_csv('data/source/movies_info_data.csv', index=False)

## Creating Sparse Matrix

In [29]:
from scipy.sparse import csr_matrix
X_sparse = csr_matrix(X)

## Training Nearest Neighbors Model

In [30]:
from sklearn.neighbors import NearestNeighbors

In [31]:
nn_model = NearestNeighbors(metric='cosine', algorithm='brute')

In [32]:
nn_model.fit(X_sparse)

NearestNeighbors(algorithm='brute', metric='cosine')

## Testing a Sample

In [33]:
test_index = np.random.choice(movies_data.shape[0])
test_index

2947

In [34]:
movies_data.iloc[test_index]

movieId                 3957
title      Billy Jack (1971)
genres          Action|Drama
Name: 2947, dtype: object

In [35]:
sample = X.iloc[test_index].values.reshape(1, -1)
sample

array([[-0.32899447,  2.66666667,  3.11350287,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ]])

In [36]:
distances, indices = nn_model.kneighbors(sample, n_neighbors = 100)

In [37]:
indices.flatten()

array([2947, 4083, 6358, 3099, 8976, 9280, 6728, 6035, 7471, 2541,  446,
       7745, 5008, 2384, 9332, 3415, 7773, 3666, 1253, 6216, 2280, 1122,
        457, 5576, 4448, 8426, 7585, 6063, 7690, 2540, 5347, 3220, 1064,
       6344, 8240, 2275, 8300, 7318, 3752, 8903, 6936, 5100, 2561, 3283,
       8375, 5822, 1811, 1188, 2347, 5664, 2176, 7509, 9324, 6336, 4237,
       7634, 6085, 7831, 1771, 5497, 7217, 8301, 9047,  439, 5947, 4257,
       3359, 4766, 5308, 6872,  341, 3920, 5475, 2560, 6295, 3311, 8059,
       4692, 7525, 6303, 8556, 3119, 5199, 7543, 6667, 3921, 8511, 8570,
       4527, 7441, 2627, 1809, 4055, 7175, 1896, 2712, 3611, 5745,   38,
       5893])

In [38]:
print('Original Movie:', movies_data.iloc[test_index]['title'], ', ID: ', movies_data.iloc[test_index]['movieId'])

Original Movie: Billy Jack (1971) , ID:  3957


In [39]:
recs = np.random.randint(0, 100, 5)
recs

array([ 9, 69, 27, 47, 71])

In [40]:
print('Recommended Movies: ')
for i in recs:
    print(f"MovieID: {movies_data.iloc[indices.flatten()[i]]['movieId']}, \
\tName: {movies_data.iloc[indices.flatten()[i]]['title']}, \
\tDistance: {distances.flatten()[i]}")

Recommended Movies: 
MovieID: 3405, 	Name: Night to Remember, A (1958), 	Distance: 0.001033944469289727
MovieID: 63222, 	Name: JCVD (2008), 	Distance: 0.023846094452369515
MovieID: 41627, 	Name: Samurai Rebellion (Jôi-uchi: Hairyô tsuma shimatsu) (1967), 	Distance: 0.0063809303037997145
MovieID: 1586, 	Name: G.I. Jane (1997), 	Distance: 0.020446382653772854
MovieID: 5521, 	Name: Principal, The (1987), 	Distance: 0.023846094452369515


In [41]:
movies_data.iloc[indices.flatten()[recs]]

,movieId,title,genres
2541,3405,"Night to Remember, A (1958)",Action|Drama
6872,63222,JCVD (2008),Action|Drama|Thriller
6063,41627,Samurai Rebellion (Jôi-uchi: Hairyô tsuma shim...,Action|Drama
1188,1586,G.I. Jane (1997),Action|Drama
3920,5521,"Principal, The (1987)",Action|Crime|Drama


## Exporting Model and Scaling Object 

In [42]:
import pickle

Exporting Model

In [43]:
with open('nn_model_movies.pkl', 'wb') as f:
    pickle.dump(nn_model, f)

Exporting Scaler

In [44]:
with open('movies_rating_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)